In [ ]:
########################################################################
# Author    : A. Alsema
# Date      : May-July 2021
# Dataset   : Visium Spatial Transcriptomics for MS lesions
# Purpose   : find cluster markers
# Required input: 
# - "3.WM.clustered.res0.2.rds" with the optimal clustering resolution
# Output    : 
# - table with thresholded (markers sig up) 
# - table with unthresholded (allmarkers) cluster markers
# - table with spot counts per cluster in cluster_contribution_WM.csv
# - table with all metadata in seurat_metadata.csv
#########################################################################

In [1]:
# load libraries
library(Seurat)
library(future)
plan("multicore", workers = 1) # to prevent multi-threading on a busy server
res = 0.2
# read in data
datasets <- readRDS(file = paste0("./RData/seurat/3.WM.clustered.res", res, ".rds"))
levels(datasets$Group)

Attaching SeuratObject

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode



[1] "CWM"       "NAWM"      "active"    "act/inact"

In [6]:
levels(datasets@active.ident)

[1] "0" "1" "2" "3" "4" "5"

In [7]:
#--------------------------  DE analyse  --------------------------
# belangrijk dat je dit op de Spatial assay doet.
DefaultAssay(datasets) <- "Spatial"
datasets$ngenes_rate <- scale(datasets$nFeature_Spatial)

markers <- Seurat::FindAllMarkers(datasets, assay='Spatial', slot='data', test.use = "MAST",
											latent.vars = c("ngenes_rate", # MAST paper shows this latent variable improves the results. https://doi.org/10.1186/s13059-015-0844-5 
                                                            "sample_name"),# *see notes below - sample-name reflects donor ID
											 logfc.threshold=0.5, only.pos=FALSE)

write.csv(markers, paste0("./Routput/Seurat/Clustering/allmarkers_lognorm_res", res, ".csv"))

# markers <- read.csv(file = paste0("Routput/allmarkers_lognorm_res 0.1.csv"))
markers <- markers[markers$p_val_adj < 0.05, ]
markers <- markers[markers$avg_log2FC > 0.5, ]
write.csv(markers, paste0("./Routput/Seurat/Clustering/markers_sig_up_lognorm_res", res, ".csv"))


Calculating cluster 0


 Completed [--------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   1% with 0 failures

 Completed [>-------------------------------------------]   2% with 0 failures

 Completed [>-------------------------------------------]   3% with 0 failures

 Completed [=>------------------------------------------]   4% with 0 failures

 Completed [=>------------------------------------------]   5% with 0 failures

 Completed [==>-----------------------------------------]   6% with 0 failures

 Completed [==>-----------------------------------------]   7% with 0 failures

 Completed [==>-----------------------------------------]   8% with 0 failures

 Completed [===>----------------------------------------]   8% with 0 failures

 Completed [===>----------------------------------------]   9% with 0 failures

 Completed [===>----------------------------------------]  10% with 0 failures

 Completed [====

In [6]:
markers <- read.csv(paste0("./Routput/Seurat/Clustering/allmarkers_lognorm_res", res, ".csv"), row.names = 1)
head(markers)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
TF,0,1.222766,0.880,0.523,0,0,TF
CNP,0,1.144472,0.951,0.620,0,0,CNP
MAG,0,1.132730,0.719,0.341,0,0,MAG
MOG,0,1.105738,0.530,0.206,0,0,MOG
PPP1R14A,0,1.097661,0.769,0.398,0,0,PPP1R14A
MARCKSL1,0,1.092514,0.802,0.436,0,0,MARCKSL1


In [7]:
# corresponds to supplemental table
markers <- markers[markers$p_val_adj < 0.05, ]
markers <- markers[abs(markers$avg_log2FC) > 0.5, ]
write.csv(markers, paste0("./Routput/Seurat/Clustering/markers_sig_log2FC=0.5_res", res, ".csv"))

In [6]:
#--------------------------  export cluster contribution for differential proportions analysis  --------------------------
cluster_contribution = table(datasets$manuscript_ID, datasets@active.ident)
cluster_contribution
write.csv(cluster_contribution, "Routput/Seurat/scCODA/cluster_contribution_WM.csv")

In [3]:
write.csv(datasets@meta.data, "Routput/Seurat/seurat_metadata.csv")

In [7]:
sessionInfo()

R version 4.2.0 (2022-04-22)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS/LAPACK: /data/bcn/p283607/anaconda3/envs/R4.2/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] SeuratObject_4.1.3 sp_1.6-0          

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.10         parallelly_1.34.0   uuid_1.1-0         
 [4] lattice_0.20-45     rlang_1.0.6         fastmap_1.1.1      
 [7] fansi_1.0.4         globals_0.16.2      tools_4.2.0        
[10] parallel_4.2.0  